In [61]:
import numpy as npy
import scipy.optimize as opt
import matplotlib.pyplot as plt

class currentProfile:
    def __init__(self, label):
        self.label=label
        self.current=[]

def dataReader(filename, label):
    inputfile=open(filename,'r')
    outputData=currentProfile(label)
    currentData=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]==str(label)):
                outputData.current.append(1e6*float(temp[2]))
        except StopIteration:
            outputData.current=npy.array(outputData.current)
            break
    return outputData

def fitFunc(time, alpha, beta, omega, gamma):
    return gamma*(1+alpha*npy.cos(2*omega*time)+beta*npy.sin(2*omega*time))

In [62]:
length=73
angles=[2.5*i/180*npy.pi for i in range(length)]
measurements=[]
monitors=[]

for i in range(length):
    measurements.append(dataReader("data/measurement.txt",int(2*2.5*i)))
    monitors.append(dataReader("data/monitor.txt",int(2*2.5*i)))

In [63]:
ratios=[]
fitParameters=[]
fitParametersCOV=[]
for i in range(length):
    minLength=720
    ratios.append(measurements[i].current[:minLength]/monitors[i].current[:minLength])
    time=npy.array([0.5*j for j in range(minLength)])
    popt,pcov=opt.curve_fit(fitFunc, time, ratios[i], p0=[-0.5,0.5,0.035,9.5])
    fitParameters.append(popt)
    fitParametersCOV.append(pcov)
    fitRatio=fitFunc(time,*popt)
    fig,axs=plt.subplots(2,1)
    axs[0].plot(time,ratios[i],label="Intensity")
    axs[0].plot(time,fitRatio,label="Fitted")
    axs[1].plot(time,(ratios[i]-fitRatio)/ratios[i],label="Residual")
    axs[1].set_xlabel("Time (s)")
    axs[0].set_ylabel("Intensity")
    axs[1].set_ylabel("Residual")
    axs[0].legend()
    axs[1].legend()
    fig.tight_layout()
    fig.savefig("figure/RatioVsTime"+str(2.5*i)+"degree.jpg")
    plt.close()

In [64]:
def alphaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.cos(2*As)-betaReal*npy.sin(2*As))/eta


def betaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.sin(2*As)+betaReal*npy.cos(2*As))/eta


def residualCurve(P, Ps, As, tanPsi, cosDelta, eta):
    return 1-alphaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2-betaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2


def fitCurve(P, Ps, As, tanPsi, cosDelta, eta):
    length=int(len(P)/3)
    P=P[:length]
    return npy.append(alphaCurve(P, Ps, As, tanPsi, cosDelta, eta), npy.append(betaCurve(P, Ps, As, tanPsi, cosDelta, eta), residualCurve(P, Ps, As, tanPsi, cosDelta, eta)))


def refractionRate(tanPsi, cosDelta):
    rho = tanPsi*(cosDelta+1j*npy.sqrt(1-cosDelta**2))
    return npy.sin(65.0/180.0*npy.pi)**2+(npy.cos(65.0/180.0*npy.pi)**2)*((1-rho)/(1+rho))**2


In [80]:
alpha=[]
alphaError=[]
beta=[]
betaError=[]
residualError=[]
partLength=58
for i in range(length):
    alpha.append(fitParameters[i][0])
    beta.append(fitParameters[i][1])
    alphaError.append(npy.sqrt(fitParametersCOV[i][0,0]))
    betaError.append(npy.sqrt(fitParametersCOV[i][1,1]))
    residualError.append(2*npy.sqrt(fitParametersCOV[i][0,0]*(fitParameters[i][0])**2+fitParametersCOV[i][1,1]*(fitParameters[i][1])**2))

anglesFinal=npy.array(angles[:partLength])
alpha=npy.array(alpha[:partLength])
beta=npy.array(beta[:partLength])
alphaError=npy.array(alphaError[:partLength])
betaError=npy.array(betaError[:partLength])
residualError=npy.array(residualError[:partLength])
residual=1-alpha**2-beta**2

# (P, Ps, As, tanPsi, cosDelta, eta)
comboX=npy.append(anglesFinal,npy.append(anglesFinal,anglesFinal))
comboY=npy.append(alpha,npy.append(beta,residual))

popt1,pcov1=opt.curve_fit(fitCurve, comboX, comboY , p0=[2,2,0,0,1.25], maxfev=10000000)
# popt2,pcov=opt.curve_fit(betaCurve, anglesFinal, beta, p0=popt1, sigma=betaError, maxfev=1000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))
# popt3,pcov=opt.curve_fit(residualCurve, anglesFinal, residual, p0=popt2, sigma=residualError, maxfev=1000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))

fig,axs=plt.subplots(3,1)
axs[0].errorbar(anglesFinal,alpha,alphaError,label="alpha")
axs[0].plot(anglesFinal,alphaCurve(anglesFinal, *popt1),label="Fit")

axs[1].errorbar(anglesFinal,beta,betaError,label="beta")
axs[1].plot(anglesFinal,betaCurve(anglesFinal, *popt1),label="Fit")

axs[2].errorbar(anglesFinal,residual,residualError,label="residual")
axs[2].plot(anglesFinal,residualCurve(anglesFinal, *popt1),label="Fit")

# axs[1].plot(angles,(ratios[i]-fitRatio)/ratios[i],label="Residual")
axs[2].set_xlabel("Angle")
axs[0].set_ylabel("alpha")
axs[1].set_ylabel("beta")
axs[2].set_ylabel("Residual")
axs[0].legend()
axs[1].legend()
axs[2].legend()
fig.tight_layout()
fig.savefig("figure/AlphaBeta.jpg",dpi=1000)
plt.close()

result=[str(popt1[i])+"±"+str(npy.sqrt(pcov1[i,i])) for i in range(len(popt1))]

print("(Ps, As, tanPsi, cosDelta, eta)",result)
print("n="+str(refractionRate(popt1[2],popt1[3])))

(Ps, As, tanPsi, cosDelta, eta) ['2.2385138986383066±0.0028254049480017263', '2.3141380074461915±0.002728902371004231', '0.9214506978751937±0.0031679760445601558', '-0.740889335009527±0.0025787866215321166', '1.138606930123005±0.0016891132796983293']
n=(-0.33079908178387507-0.28523476784758134j)


In [77]:
([1,1],[2,2])

([1, 1], [2, 2])